# CUSTOMER SEGMENTATION WITH RFM

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

####  Read the HM-Sales-2018.csv data. Make a copy of the dataframe.

In [2]:
df_ = pd.read_csv("HM-Sales-2018.csv")
df = df_.copy()

In [3]:
df.head(10)

,last_order_date,master_id,customer_value_total,order_num_total
0,11/8/2018,CG-12520,261.9600,2
1,6/12/2018,DV-13045,14.6200,2
2,10/11/2018,SO-20335,957.5775,5
3,6/9/2018,BH-11710,48.8600,7
4,4/15/2018,AA-10480,15.5520,3
5,12/5/2018,IM-15070,407.9760,3
6,11/22/2018,Sport shoes-14815,68.8100,5
7,11/11/2018,PK-19075,665.8800,6
8,5/13/2018,AG-10270,55.5000,2
9,8/27/2018,ZD-21925,8.5600,2


**Variable names**

In [4]:
df.columns

Index(['last_order_date', 'master_id', 'customer_value_total',
       'order_num_total'],
      dtype='object')

**Descriptive statistics**

In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
customer_value_total,89.0,319.931927,569.842526,1.248,29.472,95.616,294.336,3083.43
order_num_total,89.0,62.910112,131.704097,1.000,3.000,7.000,27.000,729.00


**Null Value**

In [6]:
df.isnull().sum()

last_order_date         0
master_id               0
customer_value_total    0
order_num_total         0
dtype: int64

**Variable types**

In [7]:
df.dtypes

last_order_date          object
master_id                object
customer_value_total    float64
order_num_total           int64
dtype: object

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   last_order_date       89 non-null     object 
 1   master_id             89 non-null     object 
 2   customer_value_total  89 non-null     float64
 3   order_num_total       89 non-null     int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 2.9+ KB


#### Change the type of variables that express date to date.

In [9]:
date_columns = [col for col in df.columns if "date" in col]
date_columns

['last_order_date']

In [10]:
df[date_columns] = df[date_columns].astype("datetime64[ns]")

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   last_order_date       89 non-null     datetime64[ns]
 1   master_id             89 non-null     object        
 2   customer_value_total  89 non-null     float64       
 3   order_num_total       89 non-null     int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 2.9+ KB


#### List the top 10 customers with the highest revenue.

In [12]:
df_sort = df[["master_id", "customer_value_total", "order_num_total"]]
df_sort

,master_id,customer_value_total,order_num_total
0,CG-12520,261.9600,2
1,DV-13045,14.6200,2
2,SO-20335,957.5775,5
3,BH-11710,48.8600,7
4,AA-10480,15.5520,3
...,...,...,...
84,A-KB-16315,25.9200,8
85,A-RB-19705,11.3640,1
86,A-PN-18775,193.0600,343
87,A-KD-16345,102.6240,27


In [13]:
df_sort_revenue = df_sort.sort_values(by="customer_value_total", ascending=False).head(10)
df_sort_revenue

,master_id,customer_value_total,order_num_total
14,TB-21520,3083.4300,7
73,A-BS-11590,2226.0480,512
61,A-GH-14485,2195.0880,343
58,A-EB-13870,2089.2600,27
48,A-SO-20335,1915.1550,125
53,A-PK-19075,1331.7600,216
28,BS-11590,1113.0240,8
16,GH-14485,1097.5440,7
12,EB-13870,1044.6300,3
2,SO-20335,957.5775,5


#### List the top 10 customers with the most orders.

In [14]:
df_sort_orders = df_sort.sort_values(by="order_num_total", ascending=False).head(10)
df_sort_orders

,master_id,customer_value_total,order_num_total
62,A-SN-20710,226.656,729
73,A-BS-11590,2226.048,512
79,A-JC-16105,401.968,343
56,A-KB-16585,38.920,343
49,A-BH-11710,97.720,343
86,A-PN-18775,193.060,343
61,A-GH-14485,2195.088,343
69,A-JM-15265,30.520,343
82,A-GM-14455,316.736,343
88,A-ER-13855,155.760,216


#### Calculating RFM Metrics

In [15]:
df["last_order_date"].max()

Timestamp('2018-12-27 00:00:00')

In [16]:
today_date = dt.datetime(2019, 6, 1)
today_date

datetime.datetime(2019, 6, 1, 0, 0)

In [17]:
# master_id, order_num_total, customer_value_total
rfm = df.groupby("master_id").agg({
    "last_order_date": lambda date: (today_date - date.max()).days,
    "order_num_total": lambda order: order,
    "customer_value_total": lambda price: price,
}).reset_index()

rfm.head()

,master_id,last_order_date,order_num_total,customer_value_total
0,A-AA-10480,412,27,31.104
1,A-AG-10270,384,8,111.000
2,A-BH-11710,357,343,97.720
3,A-BS-11590,178,512,2226.048
4,A-CG-12520,205,8,523.920


In [18]:
rfm.columns = ["master_id", "recency", "frequency", "monetary"]
rfm.head()

,master_id,recency,frequency,monetary
0,A-AA-10480,412,27,31.104
1,A-AG-10270,384,8,111.000
2,A-BH-11710,357,343,97.720
3,A-BS-11590,178,512,2226.048
4,A-CG-12520,205,8,523.920


#### Calculating RFM Scores

In [19]:
rfm["recency_score"] = pd.qcut(rfm["recency"], 5, labels=[5, 4, 3, 2, 1])
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first") ,5, labels=[1, 2, 3, 4, 5])
rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels=[1, 2, 3, 4, 5])

In [20]:
rfm["RFM_SCORE"] = (rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str))

In [21]:
rfm.head()

,master_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
0,A-AA-10480,412,27,31.104,1,4,2,14
1,A-AG-10270,384,8,111.000,1,3,3,13
2,A-BH-11710,357,343,97.720,2,5,3,25
3,A-BS-11590,178,512,2226.048,5,5,5,55
4,A-CG-12520,205,8,523.920,4,3,5,43


#### Segment Definition of RFM Scores

In [22]:
seg_map = {
        r'[1-2][1-2]': 'hibernating',
        r'[1-2][3-4]': 'at_Risk',
        r'[1-2]5': 'cant_loose',
        r'3[1-2]': 'about_to_sleep',
        r'33': 'need_attention',
        r'[3-4][4-5]': 'loyal_customers',
        r'41': 'promising',
        r'51': 'new_customers',
        r'[4-5][2-3]': 'potential_loyalists',
        r'5[4-5]': 'champions',
}

In [23]:
rfm["segment"] = rfm["RFM_SCORE"].replace(seg_map, regex=True)

In [24]:
rfm.head()

,master_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
0,A-AA-10480,412,27,31.104,1,4,2,14,at_Risk
1,A-AG-10270,384,8,111.000,1,3,3,13,at_Risk
2,A-BH-11710,357,343,97.720,2,5,3,25,cant_loose
3,A-BS-11590,178,512,2226.048,5,5,5,55,champions
4,A-CG-12520,205,8,523.920,4,3,5,43,potential_loyalists


#### Examine the recency, frequency and monetary averages of the segments.

In [25]:
segment_analysis =rfm.groupby("segment").agg({
    "recency": "mean",
    "frequency": "mean",
     "monetary": "mean",
    "segment": "count"
})

segment_analysis.columns = ["recency(mean)", "frequency(mean)", "monetary(mean)", "count"]
segment_analysis.head()

,recency(mean),frequency(mean),monetary(mean),count
segment,,,,
about_to_sleep,259.285714,2.285714,189.988857,7
at_Risk,387.266667,11.533333,146.875467,15
cant_loose,386.800000,248.600000,266.849200,5
champions,174.636364,224.909091,629.782909,11
hibernating,385.187500,2.500000,103.467438,16
